<a href="https://colab.research.google.com/github/SebaChamorro/gbm_learning/blob/main/02%20Analisis%20Dataset%20freMTPL2freq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from sklearn.datasets import fetch_openml
import pandas as pd

In [3]:
# Descargar el dataset de frecuencia (ID 41214 en OpenML)
df_freq = fetch_openml(data_id=41214, as_frame=True, parser='pandas').frame

# Mostrar las primeras filas y los tipos de datos
print(df_freq.head())

print(df_freq.info())

# Un pequeño análisis actuarial rápido: Frecuencia observada
frecuencia_global = df_freq['ClaimNb'].sum() / df_freq['Exposure'].sum()
print("\nFrecuencia Global de la Cartera: {frecuencia_global:.2%}")


   IDpol  ClaimNb  Exposure Area  VehPower  VehAge  DrivAge  BonusMalus  \
0    1.0        1      0.10    D         5       0       55          50   
1    3.0        1      0.77    D         5       0       55          50   
2    5.0        1      0.75    B         6       2       52          50   
3   10.0        1      0.09    B         7       0       46          50   
4   11.0        1      0.84    B         7       0       46          50   

  VehBrand     VehGas  Density Region  
0      B12  'Regular'     1217    R82  
1      B12  'Regular'     1217    R82  
2      B12   'Diesel'       54    R22  
3      B12   'Diesel'       76    R72  
4      B12   'Diesel'       76    R72  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678013 entries, 0 to 678012
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   IDpol       678013 non-null  float64 
 1   ClaimNb     678013 non-null  int64   
 2   Exposure    678013 no

In [4]:
import numpy as np

# 1. Carga de datos
print("Descargando dataset...")
df = fetch_openml(data_id=41214, as_frame=True, parser='pandas').frame

# 2. Limpieza actuarial básica
# La exposición no puede ser mayor a 1 año para este ejercicio de pricing anual
df['Exposure'] = df['Exposure'].clip(upper=1.0)
# Capeamos siniestros para evitar que un solo asegurado con 10 choques sesgue el modelo
df['ClaimNb'] = df['ClaimNb'].clip(upper=4)

# 3. Transformación de variables categóricas (Encoding)
# Usaremos Categorical dtypes para que LightGBM o XGBoost los reconozcan
col_categoricas = ['Area', 'VehBrand', 'VehGas', 'Region']

for col in col_categoricas:
    df[col] = df[col].astype('category')

# 4. Feature Engineering: Grupos de edad (DrivAge)
# Es común en México y el mundo segmentar por riesgo de juventud/vejez
df['DrivAge_Group'] = pd.cut(df['DrivAge'],
                             bins=[17, 25, 35, 45, 55, 65, 100],
                             labels=['18-25', '26-35', '36-45', '46-55', '56-65', '66+'])

# 5. Cálculo de Frecuencia Observada por grupo de edad
freq_por_edad = df.groupby('DrivAge_Group').apply(
    lambda x: x['ClaimNb'].sum() / x['Exposure'].sum(),
    include_groups=False
).reset_index(name='Frecuencia_Real')

print("\n--- Preprocesamiento Completado ---")
print(freq_por_edad)


Descargando dataset...

--- Preprocesamiento Completado ---
  DrivAge_Group  Frecuencia_Real
0         18-25         0.175076
1         26-35         0.096219
2         36-45         0.095791
3         46-55         0.103304
4         56-65         0.091599
5           66+         0.094857


/tmp/ipython-input-1852364897.py:27: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  freq_por_edad = df.groupby('DrivAge_Group').apply(


In [5]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import xgboost as xgb
from sklearn.model_selection import train_test_split

# 1. Split de datos (80% entrenamiento, 20% prueba)
train, test = train_test_split(df, test_size=0.2, random_state=42)

# Definimos las variables que usaremos (puedes añadir más)
features = ['VehPower', 'VehAge', 'DrivAge', 'BonusMalus', 'VehGas']


In [11]:
import patsy.contrasts as contr

formula = "ClaimNb ~ VehPower + VehAge + C(DrivAge_Group, contr.Treatment(reference='36-45')) + BonusMalus + VehGas"
glm_poisson = smf.glm(formula=formula,
                      data=train,
                      family=sm.families.Poisson(),
                      offset=np.log(train['Exposure'])).fit()

# Predicción del GLM (importante multiplicar por exposición al predecir n_siniestros)
test['pred_glm'] = glm_poisson.predict(test, exposure=test['Exposure'])

print(glm_poisson.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                ClaimNb   No. Observations:               542410
Model:                            GLM   Df Residuals:                   542400
Model Family:                 Poisson   Df Model:                            9
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.1432e+05
Date:                Mon, 09 Feb 2026   Deviance:                   1.7325e+05
Time:                        00:43:18   Pearson chi2:                 1.37e+06
No. Iterations:                     7   Pseudo R-squ. (CS):            0.01057
Covariance Type:            nonrobust                                         
                                                                    coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------

In [8]:
print(f"Categorías únicas de VehGas: {df['VehGas'].unique()}")
print(f"Número de categorías únicas de VehGas: {df['VehGas'].nunique()}")

Categorías únicas de VehGas: [''Regular'', ''Diesel'']
Categories (2, object): [''Diesel'', ''Regular'']
Número de categorías únicas de VehGas: 2


### Interpretación del Modelo GLM Poisson con `DrivAge_Group` (Referencia: 36-45)

La inclusión de `DrivAge_Group` en lugar de `DrivAge` permite capturar las no linealidades en la relación entre la edad del conductor y la frecuencia de siniestros. Ahora, con el grupo `36-45` como referencia, la interpretación de los coeficientes para el resto de los grupos de edad cambia significativamente.

**Análisis de los Coeficientes para `DrivAge_Group` (Referencia: 36-45):**

*   **DrivAge_Group[T.18-25] (-0.1305):** Los conductores en el grupo de edad `18-25` tienen un logaritmo de siniestros esperado 0.1305 unidades *menor* que el grupo de referencia (`36-45`). Esto sugiere una menor frecuencia de siniestros para los conductores más jóvenes en comparación con el grupo de 36-45 años.
*   **DrivAge_Group[T.26-35] (-0.2829):** Este grupo (`26-35`) tiene un logaritmo de siniestros esperado 0.2829 unidades *menor* que el grupo de referencia (`36-45`). Esto indica que los conductores en este rango de edad presentan la menor frecuencia de siniestros en comparación con el grupo de 36-45 años.
*   **DrivAge_Group[T.46-55] (0.1431):** Los conductores en este grupo (`46-55`) tienen un logaritmo de siniestros esperado 0.1431 unidades *mayor* que el grupo de referencia (`36-45`), sugiriendo un riesgo comparativamente mayor en esta franja de edad.
*   **DrivAge_Group[T.56-65] (0.0118):** Los conductores en este grupo (`56-65`) tienen un logaritro de siniestros esperado 0.0118 unidades *mayor* que el grupo de referencia (`36-45`). Sin embargo, es importante notar que el p-valor para este coeficiente es `0.563`, lo que indica que **no es estadísticamente significativo** al nivel de confianza usual (por ejemplo, 0.05). Por lo tanto, no podemos concluir que haya una diferencia significativa en la frecuencia de siniestros entre este grupo y el de 36-45 años basándonos en este modelo.
*   **DrivAge_Group[T.66+] (0.0943):** Este grupo (`66+`) tiene un logaritmo de siniestros esperado 0.0943 unidades *mayor* que el grupo de referencia (`36-45`), indicando un riesgo superior.

**Comparación con la variable `DrivAge` continua y la referencia anterior:**

Originalmente, el coeficiente de `DrivAge` era `0.0070`, sugiriendo un aumento lineal muy pequeño. Con `DrivAge_Group` y la referencia en `18-25`, observábamos que el riesgo disminuía en el grupo `26-35` y luego aumentaba para los grupos de mediana edad y mayores.

Con la referencia en `36-45`, la imagen es aún más clara:
*   Los grupos más jóvenes (`18-25` y `26-35`) muestran un riesgo significativamente *menor* que el grupo `36-45`.
*   El grupo `26-35` es el que presenta el menor riesgo en comparación con la referencia `36-45`.
*   El grupo `46-55` tiene el riesgo significativamente *mayor* en comparación con `36-45`.
*   Los grupos `56-65` no muestran una diferencia significativa con `36-45`.
*   El grupo `66+` muestra un riesgo significativamente *mayor* que `36-45`.

**Implicaciones Clave:**

*   **Patrón de Riesgo no Lineal:** Este análisis refuerza la conclusión de que la relación entre la edad del conductor y la frecuencia de siniestros no es lineal. El riesgo parece ser menor para los grupos más jóvenes en comparación con la mediana edad (`36-45`), alcanzando un pico en el grupo `46-55` y luego manteniéndose alto (o con una tendencia a ser mayor) en la vejez.
*   **Claridad en la Comparación:** Al elegir un grupo de referencia central (`36-45`), las comparaciones de riesgo para los grupos más jóvenes y más viejos se vuelven más intuitivas.
*   **Identificación de Grupos Clave:** Este ajuste nos permite identificar que los grupos `26-35` tienen un riesgo considerablemente bajo, mientras que los grupos `46-55` y `66+` muestran un riesgo elevado en comparación con el grupo de mediana edad.

Este análisis con `DrivAge_Group` y una referencia central proporciona una visión muy realista y matizada de la relación entre la edad y el riesgo de siniestros, lo cual es fundamental para una tarificación de seguros precisa.